In [3]:
import telebot
from telebot import types
import requests
import wget
import numpy as np
import openpyxl as op

import pandas as pd
import sqlite3 as sl
import os

import unicodedata # для игнора регистра


TOKEN = "6703792810:AAHn1tEMb6oeEpSVEocOr4J9j0WIdTGpwqM"
bot = telebot.TeleBot(TOKEN)



@bot.message_handler(commands=['start', 'hello']) # приветствие
def send_welcome(message):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    bot.reply_to(message, "Здравствуй, " + message.chat.first_name + "!", parse_mode='html', reply_markup=markup)

@bot.message_handler(commands=['help'])
def help(message):
    bot.reply_to(message, "/upload_excel - обновить базу данных\n/change_password - смена пароля администратора\n/dataset_info - работа с базой данных")

@bot.message_handler(commands=['upload_excel']) # загрузка таблицы
def upload_excel(message):
    markup = types.ReplyKeyboardRemove()
    sent = bot.send_message(message.chat.id, 'Введите пароль', reply_markup=markup)
    bot.register_next_step_handler(sent, tab_login)

@bot.message_handler(commands=['change_password']) # смена пароля
def change_password(message):
    markup = types.ReplyKeyboardRemove()
    sent = bot.send_message(message.chat.id, 'Введите пароль', reply_markup=markup)
    bot.register_next_step_handler(sent, pass_login)

def pass_login(message):
    password = pass_reader()
    if message.text == password:
        markup = types.ReplyKeyboardRemove()
        sent = bot.send_message(message.chat.id, 'Успешный вход. Введите новый пароль')
        bot.register_next_step_handler(sent, pass_writer)
    else:
        bot.send_message(message.chat.id, 'Пароль неверен')

def tab_login(message):
    password = pass_reader()
    if message.text == password:
        markup = types.ReplyKeyboardRemove()
        sent = bot.send_message(message.chat.id, 'Успешный вход. Загрузите таблицу')
        bot.register_next_step_handler(sent, get_file)
    else:
        bot.send_message(message.chat.id, 'Пароль неверен')

def get_file(message):
    if message.content_type == 'document':
      URL = bot.get_file_url(message.document.file_id)  # получаем ссылку на документ
      URL_parts = URL.split(".") # дробим ссылку по точкам, чтобы последним элементом списка было расширение файла

      if URL_parts[-1] == 'xlsx': # если расширение xlsx -- бот загружает таблицу
        folder_path = os.getcwd()
        excel_file = os.path.join(folder_path, 'data.xlsx')
        db_file = os.path.join(folder_path, 'data.db')

        if os.path.exists(excel_file): # для исключения создания копий, удаляем старую версию БД, если такая существует
          bot.send_message(message.chat.id, "Excel-файл существует")
          os.remove(excel_file)
          bot.send_message(message.chat.id, "Существующий Excel-файл удален")

        if os.path.exists(db_file): # для исключения создания копий, удаляем старую версию БД, если такая существует
          bot.send_message(message.chat.id, "db-файл существует")
          os.remove(db_file)
          bot.send_message(message.chat.id, "Существующий db-файл удален")

        wget.download(URL, 'data.xlsx')

        convertation('data.xlsx')

        bot.send_message(message.chat.id, "Таблица загружена")

      else: # если расширение НЕ xlsx -- бот пишет что недопустимый тип файла
        bot.send_message(message.chat.id, "Недопустимый тип файла. Загрузите таблицу в формате .xlsx")
    else:
      bot.send_message(message.chat.id, "Загруженный файл не является документом")

def convertation(file_name):
  folder_path = os.getcwd() # путь к корневой папке
  excel_file = os.path.join(folder_path, file_name) # добавляем путь к файлу
  if os.path.exists(excel_file): # проверяем, что файл по указанному пути существует
    Empty_Rows_Exterminator(excel_file)
    xlsx = pd.ExcelFile(excel_file) # переводим xlsx в pd формат (DataFrame object)
    sheet_name = 'СВОД'
    db = Table_UnRegister(xlsx.parse(sheet_name))
    db = db.rename(columns={'Субъект Российской Федерации': 'Region'})
    db_file = os.path.join(folder_path, 'data.db') # указываем путь для сохранения нового файла
    con = sl.connect(db_file) # создание подключения к базе данных
    cur = con.cursor() # создание объекта "курсор"
    db.to_sql('INV_Table', con, index = False) # , if_exist = 'replace' # собственно конвертация из DataFrame в SQL
    con.commit() # сохранение изменений
    con.close() # закрытие соединения
    xlsx.close() # закрытие файла, иначе перезапись не работает и код ложится отдыхать

  else:
      print(f"Excel-файл '{excel_file}' не существует") # иначе говорим, что Excel файла не существует

def pass_reader():
   f = open('password.txt','r')
   str = f.readline()
   return str

def pass_writer(message):
   password = message.text
   f = open('password.txt','w')
   f.truncate(0)
   f.write(password)
   markup = types.ReplyKeyboardRemove()
   bot.send_message(message.chat.id, 'Пароль изменен!', reply_markup=markup)

def Empty_Rows_Exterminator(excel_path):
   if __name__ == '__main__':
      book = op.load_workbook(excel_path)
      sheet = book['СВОД']
      for row in sheet:
         row_checker(sheet,row)
   book.save(excel_path)


def row_checker(sheet, row):
    for cell in row:
        if cell.value != None:
              return
    sheet.delete_rows(row[0].row, 1)





@bot.message_handler(commands=['dataset_info']) #работа с БД
def dataset_info(message):
    markup = types.ReplyKeyboardRemove()
    sent = bot.send_message(message.chat.id, 'Введите полное наименование субъекта Российской Федерации', reply_markup=markup)
    bot.register_next_step_handler(sent, data_work)

def data_work(message):
    markup = types.ReplyKeyboardRemove()
    con = sl.connect('data.db')
    user_message = UnRegister(str(message.text))
    try:
      with con:
        data = con.execute('SELECT * FROM INV_Table WHERE Region = :region_;', {'region_': user_message})
        for row in data:
          # dict.fromkeys(seq[, value]) -- можно попробовать
            dict = {
              'Субъект Российской Федерации' : row[1],
              'Закон ИНВ' : row[2],
              'ИНВ для телекома' : row[3],
              'Максимальный размер вычета' : row[4],
              'Ставка для исчисления предельной величины' : row[5],
              'Участие в программе увеличение производительности труда' : row[6],
              'Инвестпроект' : row[7],
              'ОКВЭД 61' : row[8],
              '2022' : row[9],
              '2023' : row[10],
              '2024' : row[11]
              }
      con.close()

      if dict.get('ИНВ для телекома') == 'НЕТ':
        markup = types.ReplyKeyboardRemove()
        bot.send_message(message.chat.id, 'Для данного региона нет ИНВ', reply_markup=markup)
        bot.send_message(message.chat.id, 'Вы вернулись на основную страницу')
      else:
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
        but1 = types.KeyboardButton("Выйти")
        but3 = types.KeyboardButton("Закон ИНВ")
        but4 = types.KeyboardButton("ИНВ для телекома")
        but5 = types.KeyboardButton("Максимальный размер вычета")
        but6 = types.KeyboardButton("Ставка для исчисления предельной величины")
        but7 = types.KeyboardButton("Участие в программе увеличение производительности труда")
        but8 = types.KeyboardButton("Инвестпроект")
        but9 = types.KeyboardButton("ОКВЭД 61")
        but10 = types.KeyboardButton("2022")
        but11 = types.KeyboardButton("2023")
        but12 = types.KeyboardButton("2024")
        markup.add(but1, but3, but4, but5, but6, but7, but8, but9, but10, but11, but12)
        sent = bot.send_message(message.chat.id, 'Чего хотите?', reply_markup=markup)
        bot.register_next_step_handler(sent, inform, dict)
    except:
      markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
      but1 = types.KeyboardButton("Выйти")
      but2 = types.KeyboardButton("Попробовать снова")
      markup.add(but1, but2)
      sent = bot.send_message(message.chat.id, 'Регион с таким названием не найден', reply_markup=markup)
      bot.register_next_step_handler(sent, try_again)

def UnRegister(text):
    return unicodedata.normalize("NFKD", text.casefold())

def Table_UnRegister(excel_file):
  counter=0
  while str(excel_file.loc[counter, 'Субъект Российской Федерации']) != 'nan':
    region = str(excel_file['Субъект Российской Федерации'][counter])
    region_normalize = UnRegister(region)
    excel_file.loc[counter, 'Субъект Российской Федерации'] = region_normalize
    counter=counter+1
  return excel_file



def inform(message, dict):
      if message.text == 'Выйти':
        markup = types.ReplyKeyboardRemove()
        bot.send_message(message.chat.id, "Вы вернулись на основную страницу", parse_mode='html', reply_markup=markup)
      else:
        if dict.get(message.text) == None:
          sent = bot.send_message(message.chat.id, "Нет информации")
          bot.register_next_step_handler(sent, inform, dict)
        else:
          sent = bot.send_message(message.chat.id, dict.get(message.text))
          bot.register_next_step_handler(sent, inform, dict)

def try_again(message):
  markup = types.ReplyKeyboardRemove()
  if message.text == 'Выйти':
    bot.send_message(message.chat.id, "Вы вернулись на основную страницу", parse_mode='html', reply_markup=markup)
  elif message.text == 'Попробовать снова':
    sent = bot.send_message(message.chat.id, 'Введите полное наименование субъекта Российской Федерации', reply_markup=markup)
    bot.register_next_step_handler(sent, data_work)
  else:
    bot.send_message(message.chat.id, "В любом случае, вы вернулись на основную страницу", parse_mode='html', reply_markup=markup)

bot.polling(non_stop=True)